In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import io
import regex

# Upload Data

In [4]:
from google.colab import files

def upload_csv(**kwargs):
    file_dict = files.upload()
    file_path = io.BytesIO(file_dict[next(iter(file_dict))])
    return pd.read_csv(file_path, index_col=0, **kwargs)

In [5]:
df = upload_csv()

Saving original_data.csv to original_data.csv


In [6]:
tweet_data = df.copy()
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 1956967341 to 1753919049
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  40000 non-null  object
 1   content    40000 non-null  object
dtypes: object(2)
memory usage: 937.5+ KB


In [7]:
pd.unique(tweet_data['sentiment'])

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [8]:
tweet_data.loc[tweet_data["sentiment"]=="empty", "sentiment"] = "sadness"
tweet_data.loc[tweet_data["sentiment"]=="boredom", "sentiment"] = "neutral"
tweet_data.loc[tweet_data["sentiment"]=="hate", "sentiment"] = "anger"
tweet_data.loc[tweet_data["sentiment"]=="fun", "sentiment"] = "enthusiasm"
pd.unique(tweet_data['sentiment'])

array(['sadness', 'enthusiasm', 'neutral', 'worry', 'surprise', 'love',
       'anger', 'happiness', 'relief'], dtype=object)

In [9]:
# Define identifiers
urls = r"http:[^\s]+|w{3}\.[^\s]+|[\w]+\.[\w]{3}[/\w]*"
users = r"@(\w){1,}"
tags = r"#(\w){1,}"
symbols = r"[&[\w]+;|~$/%/^/&/*-/+/=/\/|///?:;\"<>,]|[.]+"

# Parse unique identifiers
tweet_data['num_urls'] = tweet_data['content'].apply(lambda text: len(regex.findall(urls, text.lower()))).values
tweet_data['content_parsed'] = tweet_data['content'].apply(lambda text: regex.sub(urls, "", text)).values
tweet_data['num_pings'] = tweet_data['content_parsed'].apply(lambda text: len(regex.findall(users, text.lower()))).values
tweet_data['content_parsed'] = tweet_data['content_parsed'].apply(lambda text: regex.sub(users, "", text)).values
tweet_data['num_tags'] = tweet_data['content_parsed'].apply(lambda text: len(regex.findall(tags, text.lower()))).values
tweet_data['content_parsed'] = tweet_data['content_parsed'].apply(lambda text: regex.sub(tags, "", text)).values
tweet_data['#!'] = tweet_data['content_parsed'].str.count('!')
tweet_data['#?'] = tweet_data['content_parsed'].str.count('\\?')
tweet_data['#...'] = tweet_data['content_parsed'].str.count('\\.{2,}')
tweet_data['Upper/Lower ratio'] = (tweet_data['content_parsed'].str.findall(r'[A-Z]').str.len()+1) / (tweet_data['content_parsed'].str.findall(r'[a-z]').str.len()+1)
tweet_data['Number of Words'] = tweet_data['content_parsed'].apply(lambda n: len(n.split()))
tweet_data['content_parsed'] = tweet_data['content_parsed'].apply(lambda text: text.replace(';', '; '))
tweet_data['content_parsed'] = tweet_data['content_parsed'].apply(lambda text: text.replace('&', ' &'))
tweet_data['content_parsed'] = tweet_data['content_parsed'].apply(lambda text: regex.sub(symbols, " ", text)).values
tweet_data['content_parsed'] = tweet_data['content_parsed'].apply(lambda text: regex.sub(r"'", "", text)).values

In [10]:
words = r"[\w]+"
tweet_data['words'] = tweet_data['content_parsed'].apply(lambda text: regex.findall(words, text.lower())).values
tweet_data["avg word length"] = tweet_data['words'].apply(lambda word_array: np.mean([len(word) for word in word_array]))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [11]:
tweet_data = tweet_data.drop(columns=['content_parsed'])

In [12]:
tweet_data.head(10)

,sentiment,content,num_urls,num_pings,num_tags,#!,#?,#...,Upper/Lower ratio,Number of Words,words,avg word length
tweet_id,,,,,,,,,,,,
1956967341,sadness,@tiffanylue i know i was listenin to bad habi...,0,1,0,0,0,0,0.016129,17,"[i, know, i, was, listenin, to, bad, habit, ea...",3.812500
1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,0,0,0,0,0,2,0.045455,10,"[layin, n, bed, with, a, headache, ughhhh, wai...",4.000000
1956967696,sadness,Funeral ceremony...gloomy friday...,0,0,0,0,0,2,0.074074,3,"[funeral, ceremony, gloomy, friday]",6.750000
1956967789,enthusiasm,wants to hang out with friends SOON!,0,0,0,1,0,0,0.192308,7,"[wants, to, hang, out, with, friends, soon]",4.142857
1956968416,neutral,@dannycastillo We want to trade with someone w...,0,1,0,0,0,0,0.054545,14,"[we, want, to, trade, with, someone, who, has,...",4.000000
1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...,0,1,0,0,1,0,0.080000,15,"[re, pinging, why, didnt, you, go, to, prom, b...",3.466667
1956968487,sadness,"I should be sleep, but im not! thinking about ...",0,0,0,3,0,0,0.031579,26,"[i, should, be, sleep, but, im, not, thinking,...",3.760000
1956968636,worry,Hmmm. http://www.djhero.com/ is down,1,0,0,0,0,0,0.200000,3,"[hmmm, is, down]",3.333333
1956969035,sadness,@charviray Charlene my love. I miss you,0,1,0,0,0,0,0.142857,6,"[charlene, my, love, i, miss, you]",3.666667


In [14]:
tweet_data.to_csv("cleaned_data.csv")